# CNN Cancer Detection Kaggle Mini Project - Histopathological Cancer Detection Project

## Introduction
In this project, we focus on the detection of metastatic cancer in histopathologic scans of lymph node sections. The dataset used for this task is sourced from Kaggle's "Histopathologic Cancer Detection" challenge, which aims to identify metastatic tissue in various scans.

## Data Analysis and Preprocessing
### Analyzing and Displaying Images
We begin by analyzing the images available in the dataset. By visualizing different samples, we gain insights into the various characteristics and features present within the histopathologic scans.

### Image Augmentation
To enhance the diversity of the dataset and improve the model's generalization capabilities, we apply various image augmentation techniques. These may include rotations, flipping, scaling, and other transformations that enrich the dataset without altering the underlying information.

### Splitting the Dataset
The dataset is then divided into two main parts: training and validation sets. This split ensures that we have a separate set of data to evaluate the model's performance, independent of the data used to train the model.

## Model Training
### Convolutional Neural Network (CNN)
We employ a Convolutional Neural Network (CNN) for the task of detecting metastatic cancer in the histopathologic scans. The CNN architecture is particularly well-suited for image recognition tasks, as it can capture hierarchical patterns in the data.

The training process involves feeding the augmented and preprocessed images into the network, allowing it to learn the intricate patterns associated with metastatic tissues. Regular evaluations on the validation set provide insights into the model's accuracy and performance, guiding further optimizations.

## Conclusion
The project showcases the application of deep learning, specifically CNNs, in the critical task of cancer detection through histopathologic scans. By applying robust data analysis, preprocessing, and training methodologies, we strive to create a model that can effectively identify metastatic tissues, contributing to early and accurate diagnosis.



In [ ]:
%reset -f
#!pip uninstall torch -y
#!pip uninstall torchvision -y
#!pip install torch==1.10.1 torchvision==0.11.2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.utils.model_zoo as model_zoo
from sklearn.metrics import roc_curve, auc
import torchvision
from sklearn.model_selection import StratifiedShuffleSplit

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import random
import copy
import os

# Check if gpu support is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline

csv_submission_ex_file = '../input/sample_submission.csv'
csv_file = '../input/train_labels.csv'
train_dir = '../input/train/'
test_dir = '../input/test/'

# Any results you write to the current directory are saved as output.

# CNN Model
- A powerful pre-trained model is ResNeXt-50 with a 32x4d configuration, which can be easily accessed through the PyTorch library. Here, we'll explore how to customize this model for a specific task and then deploy it on the device for inference.
- Loading the Pre-trained Model: First, we load the pre-trained ResNeXt-50 model. This  fetches the pre-trained weights of the ResNeXt-50 model, which has been trained on a vast dataset )Imagenet), allowing it to recognize various features in images. 
- Often, the pre-trained model's final layer doesn't match the number of classes in our specific task. In our case, we want to modify it to output just one (binary) value since we are using a sigmoid. 



In [ ]:
# Load the pretrained model
model_resnext50_32x4d = torchvision.models.resnext50_32x4d(pretrained=True)

# Replace the last fully connected layer to match the number of classes you want (in this case, 1)
num_features = model_resnext50_32x4d.fc.in_features
model_resnext50_32x4d.fc = nn.Linear(num_features, 1)

# Create a sequential model with the modified ResNet and a sigmoid activation
model = nn.Sequential(model_resnext50_32x4d, nn.Sigmoid())

# Move the model to the device
model.to(device)


In [ ]:
csv_pd = pd.read_csv(csv_file)   
csv_pd.describe()

# NormFinderDataset 
Is a subclass of PyTorch's Dataset class. This class is designed to handle a dataset where information about the images is stored in a CSV file, and the images themselves are located in a specified directory. 
Here's a detailed description of the class:

Initialization (__init__ method)
The class is initialized with three parameters:

csv_file: The path to the CSV file containing information about the images.
img_dir: The directory where the image files are located.
transform: An optional transformation that can be applied to the images (e.g., resizing, normalization).
Inside the initialization method, the CSV file is read into a Pandas DataFrame, and the image directory and transformation are stored as attributes of the class.

Length Method (__len__ method)
The __len__ method returns the number of samples in the dataset. It does this by returning the length of the DataFrame that was created from the CSV file. This method allows the use of Python's built-in len() function on an instance of the class to get the number of samples.

Get Item Method (__getitem__ method)
The __getitem__ method is used to retrieve a specific sample from the dataset given an index idx. Here's how it works:

It constructs the image file name by joining the image directory with the corresponding entry from the CSV file and adding the '.tif' extension.
It opens the image file using the Python Imaging Library (PIL) Image.open method.
If a transformation was provided during initialization, it is applied to the image.
Finally, it returns a dictionary containing the transformed image sample.

In [ ]:
class NormFinderDataset(torch.utils.data.dataset.Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.csv_file = pd.read_csv(csv_file)
        self.img_dir = img_dir

        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.csv_file.iloc[idx, 0])
        img_name = img_name + '.tif'

        sample = Image.open(img_name)

        if self.transform is not None:
            sample = self.transform(sample)

        return {'sample': sample}

# Defining Transformations
- First, a series of transformations is defined using torchvision.transforms.Compose. 
- These transformations include resizing the images to 224x224 pixels and converting them to PyTorch tensors

# Creating Datasets and DataLoaders
- Next, the code creates two datasets for testing and training using the NormFinderDataset class, applying the transformations defined earlier. 
- Corresponding DataLoaders are also created with a batch size of 1024 and no shuffling:

In [ ]:
normfinder_transformations = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(),
])

normfinder_test_dataset = NormFinderDataset(csv_submission_ex_file, test_dir, normfinder_transformations)
normfinder_test_dataloader = torch.utils.data.DataLoader(normfinder_test_dataset, batch_size=1024, shuffle=False)

normfinder_train_dataset = NormFinderDataset(csv_file, train_dir, normfinder_transformations)
normfinder_train_dataloader = torch.utils.data.DataLoader(normfinder_train_dataset, batch_size=1024, shuffle=False)

pop_mean = []
pop_std0 = []
for data in tqdm(normfinder_test_dataloader, 0):
    # shape (batch_size, 3, height, width)
    numpy_image = data['sample'].numpy()

    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0, 2, 3))
    batch_std0 = np.std(numpy_image, axis=(0, 2, 3))

    pop_mean.append(batch_mean)
    pop_std0.append(batch_std0)

for data in tqdm(normfinder_train_dataloader, 0):
    # shape (batch_size, 3, height, width)
    numpy_image = data['sample'].numpy()

    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0, 2, 3))
    batch_std0 = np.std(numpy_image, axis=(0, 2, 3))

    pop_mean.append(batch_mean)
    pop_std0.append(batch_std0)

# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
pop_mean = np.array(pop_mean).mean(axis=0)
pop_std0 = np.array(pop_std0).mean(axis=0)

print('mean: {}'.format(pop_mean))
print('std0: {}'.format(pop_std0))

# Data Transformations

- The code snippet defines two sets of image transformations using PyTorch's torchvision library. 
- These transformations are commonly used to preprocess images before feeding them into a deep learning model.

- The first set of transformations, includes the following steps:

    - Random Horizontal Flip: Randomly flips the image horizontally.
    - Random Vertical Flip: Randomly flips the image vertically.
    - Resize to 224: Resizes the image to 224x224 pixels, a common input size for models like ResNet.
    - Convert to Tensor: Converts the image to a PyTorch tensor.
    - Normalize: Normalizes the image using given mean and standard deviation values for each color channel.

In [ ]:
# Let's define some transformations for the input data, crop to 64px and then resize to 224 to fit resnet input size
data_transformations = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.70017236, 0.5436771, 0.6961061], std=[0.22246036, 0.26757348, 0.19798167]),
])

# Let's define some transformations for the test data, crop to 32px and then resize to 224 to fit resnet input size
test_transformations = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.70017236, 0.5436771, 0.6961061], std=[0.22246036, 0.26757348, 0.19798167]),
])

In [ ]:
# let's see some negative images
fig=plt.figure(figsize=(16, 16))

samples_per_type = 3
negative_found = 0

while negative_found < samples_per_type:
    idx = random.randint(0, len(csv_pd))
    # Negatives
    if csv_pd.iloc[idx, 1] == 0:
        negative_found = negative_found + 1
        image = Image.open(train_dir + csv_pd.iloc[idx, 0] + '.tif')
        fig.add_subplot(2, 5, negative_found)
        plt.title('Negative Label')
        plt.imshow(image)
        
        image = data_transformations(image)
        back_transform = torchvision.transforms.ToPILImage()
        image = back_transform(image)
        fig.add_subplot(2, 5, negative_found + samples_per_type)
        plt.title('Transformed')
        plt.imshow(image)

plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0)   
plt.show()    


# let's see some positive images
fig2=plt.figure(figsize=(16, 16))

positive_found = 0
        
while positive_found < samples_per_type:
    idx = random.randint(0, len(csv_pd))
    # Positives
    if csv_pd.iloc[idx, 1] == 1:
        positive_found = positive_found + 1
        image = Image.open(train_dir + csv_pd.iloc[idx, 0] + '.tif')
        fig2.add_subplot(2, 5, positive_found)
        plt.title('Positive Label')
        plt.imshow(image)
                
        image = data_transformations(image)
        back_transform = torchvision.transforms.ToPILImage()
        image = back_transform(image)
        fig2.add_subplot(2, 5, positive_found + samples_per_type)
        plt.title('Transformed')
        plt.imshow(image)

plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0)   
plt.show()

# Visualization
- The code snippet is a visualization tool that displays a side-by-side comparison of original and transformed images from a test directory. 



In [ ]:
# let's see some images after transform.
fig=plt.figure(figsize=(16, 16))

samples = 5

for i in range(samples):
    random_file = random.choice(os.listdir(test_dir))
    image = Image.open(test_dir + random_file)
    fig.add_subplot(2, 5, i + 1)
    plt.title('Original')
    plt.imshow(image)
    
    image = test_transformations(image)
    back_transform = torchvision.transforms.ToPILImage()
    image = back_transform(image)
    fig.add_subplot(2, 5, i + 1 + samples)
    plt.title('Transformed')
    plt.imshow(image)

plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0)   
plt.show()

In [ ]:
# Dataset Class for reading the data ids and labels from the CSV file.
# From the CSV file only the specified indexes are saved in this dataset(specified indexes after the train/val split)

class HPLCDDataset(torch.utils.data.dataset.Dataset):
    """HPLCDDataset dataset."""

    def __init__(self, csv_file, img_dir, idxs, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            img_dir (string): Directory with all the images.
            idxs (list): List with indexes.
            transform (callable, optional): Optional transform to be applied
                on a sample
        """
        self.csv_file = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.idxs = idxs

        self.transform = transform

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.csv_file.iloc[self.idxs[idx], 0])
        img_name = img_name + '.tif'

        sample = Image.open(img_name)
        target = self.csv_file.iloc[self.idxs[idx], 1]

        if self.transform is not None:
            sample = self.transform(sample)

        return sample, target
    
    def __getlabel__(self, idx):
        return self.csv_file.iloc[self.idxs[idx], 1]

# Data preparation
- The provided code snippet sets up the data preparation for training a deep learning model using PyTorch. 
- It includes the creation of training and validation datasets and corresponding data loaders. Here's a detailed breakdown of the code.
- The code starts by defining the batch size, which determines how many images will be processed together in each iteration of training.
- Next, the code creates two instances of a custom dataset class, HPLCDDataset, for the training and validation sets. The datasets are created using the previously defined transformations, data_transformations for training and test_transformations for validation
-  Data loaders are then created for both the training and validation datasets. These data loaders handle batching, shuffling, and other data loading details.

In [ ]:
# Use a batch size of 256 images
batch_size = 8

# Split data 10% validation 90% train with balanced data between the two datasets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=7)

# Indexes in the CSV file for both train/val datasets
train_index, test_index = next(sss.split(csv_pd["id"], csv_pd["label"]))

train_dataset = HPLCDDataset(csv_file, train_dir, train_index, data_transformations)
test_dataset = HPLCDDataset(csv_file, train_dir, test_index, test_transformations)

# Create loders for training/validation sets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

dataloaders = {'train': train_loader, 'test': test_loader}

In [ ]:
# Lets see how the data is balanced in the two Datasets
train_positive_labels = 0
train_negative_labels = 0
for index in range(train_dataset.__len__()):
    label = train_dataset.__getlabel__(index)
    if label == 0:
        train_negative_labels = train_negative_labels + 1
    else:
        train_positive_labels = train_positive_labels + 1

# Lets see how the data is balanced in the two Datasets
test_positive_labels = 0
test_negative_labels = 0
for index in range(test_dataset.__len__()):
    label = test_dataset.__getlabel__(index)
    if label == 0:
        test_negative_labels = test_negative_labels + 1
    else:
        test_positive_labels = test_positive_labels + 1

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Negative', 'Positive'

fig, axs = plt.subplots(1, 2)
plt.subplots_adjust(right=1.5)

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.3f}%\n({v:d})'.format(p=pct,v=val)
    return my_autopct

axs[0].pie([train_negative_labels, train_positive_labels], labels=labels, autopct=make_autopct([train_negative_labels, train_positive_labels]),
        shadow=True, startangle=90)
axs[0].axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
axs[0].set_title('Train Dataset\n Samples:{}'.format(train_negative_labels + train_positive_labels))

axs[1].pie([test_negative_labels, test_positive_labels], labels=labels, autopct=make_autopct([test_negative_labels, test_positive_labels]),
        shadow=True, startangle=90)
axs[1].axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
axs[1].set_title('Test Dataset\n Samples:{}'.format(test_negative_labels + test_positive_labels))

plt.show()

# Main training loop

- The train function is a standard training loop for a deep learning model using PyTorch. Here's a brief overview:

- Parameters
model: The neural network model to be trained.
criterion: The loss function used to evaluate the model.
optimizer: The optimization algorithm used to update the model's weights.
scheduler: The learning rate scheduler to adjust the learning rate during training.
dataloaders: Data loaders for training and testing datasets.
logger: A logging utility to record training progress.
- Initialization
Initializes variables to track training and testing loss and accuracy.
Training and Testing Loop
Iterates through both training and testing phases for each epoch:
Training Phase:
Sets the model to training mode.
Computes gradients and updates the model's weights.
- Testing Phase:
Sets the model to evaluation mode.
Makes predictions without updating the model's weights.
- Statistics:
Computes and stores loss and accuracy during both phases.
Logging:
Logs results if in the testing phase.
- Return Values
Returns the trained model, updated optimizer, scheduler, test output, and logger.
- Additional Notes
The code is structured for binary classification, using a threshold of 0.5 to determine class labels.
Includes progress tracking using a progress bar (tqdm).

In [ ]:
def train(model, criterion, optimizer, scheduler, dataloaders, logger):
    
    dataset_sizes = {'train': len(dataloaders['train']),
                     'test': len(dataloaders['test'])}
    
    train_loss = 0.0
    train_acc = 0.0
    test_loss = 0.0
    test_acc = 0.0

    # Each epoch has a training and validation phase
    for phase in ['train', 'test']:
        if phase == 'train':
            scheduler.step()
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0
            
        test_output_results = []
        test_output_expected = []

        pbar = tqdm(enumerate(dataloaders[phase]))
        # Iterate over data.
        for i, (inputs, labels) in pbar:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels.float().view(dataloaders[phase].batch_size, 1))
                                
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
            # statistics
            running_loss += loss.item()
            
            preds = torch.where(outputs > 0.5, torch.Tensor([1]).to(device), torch.Tensor([0]).to(device))
            running_corrects += torch.sum(preds == labels.float().view(dataloaders[phase].batch_size, 1)).item()
                
            if phase == 'test':
                test_output_results = np.concatenate([test_output_results, outputs.view(-1).cpu().numpy()])
                test_output_expected = np.concatenate([test_output_expected, labels.view(-1).cpu().numpy()])

            pbar.set_description('[{} {}/{}] Loss: {:.4f}, Acc: {:.4f}'.format(phase, i, dataset_sizes[phase],
                running_loss / (i+1), running_corrects / ((i+1) * dataloaders[phase].batch_size)))

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects / dataset_sizes[phase] / dataloaders[phase].batch_size

        if phase == 'train':
            train_loss = epoch_loss
            train_acc = epoch_acc

        if phase == 'test':
            test_loss = epoch_loss
            test_acc = epoch_acc
            logger.append([train_loss, train_acc, test_loss, test_acc])
            
        test_output = {'expected': test_output_expected, 'results': test_output_results}

    return model, optimizer, scheduler, test_output, logger

In [ ]:
def plot_results(logger):
    plt.plot(logger)
    plt.xlabel('Epoch')
    plt.grid(True)
    plt.legend(['train loss', 'train accuracy', 'validation loss', 'validation accuracy'])
    plt.show()

In [ ]:
def plot_roc_auc(test_output):        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()   
    fpr, tpr, _ = roc_curve(test_output['expected'], test_output['results'])
    roc_auc = auc(fpr, tpr)
   
    plt.subplot(121, aspect='equal')
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.5f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    
    negative_results = []
    positive_results = []
    for idx, (res) in enumerate(test_output['expected']):
        if res == 1:
            positive_results.append(test_output['results'][idx])
        elif res == 0:
            negative_results.append(test_output['results'][idx])
        else:
            print('ERROR HIST!!!')
    
    bins = np.linspace(min(test_output['results']), max(test_output['results']), 100)
    
    plt.subplot(122)
    plt.hist(positive_results, bins, alpha=0.5, label='Positive', histtype='step')
    plt.hist(negative_results, bins, alpha=0.5, label='Negative', histtype='step')
    plt.yscale('log')
    plt.legend(loc='upper center')
    plt.grid(True)
    
    plt.subplots_adjust(bottom=0.0, right=2.2, top=1)    
    plt.show() 

# The code snippet sets up the optimization, loss function, learning rate scheduler, and logging for training a deep learning model using PyTorch.
- Adam Optimizer: Utilizes the Adam optimization algorithm with a learning rate of 0.001
- Binary Cross-Entropy Loss: Used for binary classification tasks
- Step Learning Rate Scheduler: Decays the learning rate by a factor of 0.1 every 3 epochs 
- Load Checkpoint Option: Includes an option to load a previously saved checkpoint. If load_checkpoint is set to True, the model, optimizer, previous epochs, data loaders, and logger are loaded from the checkpoint file.

In [ ]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)
criterion = nn.BCELoss()

# Decay LR by a factor of 0.1 every x epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# define the global logger
logger = [[0.45, 0.55, 0.45, 0.55]]

previous_epochs = 0

load_checkpoint = False
if load_checkpoint == True:
    checkpoint = torch.load("checkpoint_ep1.plt")
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    previous_epochs = checkpoint['epoch']
    dataloaders = checkpoint['dataloaders']
    logger = checkpoint['logger']

In [ ]:
epochs = 10

for i in range(epochs):
    print('Epoch: {}/{}'.format(i + previous_epochs + 1, epochs + previous_epochs))
    model, optimizer, exp_lr_scheduler, test_output, logger = train(model, criterion, optimizer, exp_lr_scheduler, dataloaders, logger)

    #SAVE
    #torch.save({
    #        'epoch': i + 1 + previous_epochs,
    #        'model_state_dict': model.state_dict(),
    #        'optimizer_state_dict': optimizer.state_dict(),
    #        'logger': logger,
    #        'dataloaders': dataloaders
    #        }, "checkpoint_ep{}.plt".format(i + 1 + previous_epochs))
    
    clear_output()

In [ ]:
plot_roc_auc(test_output) 
plot_results(logger)

def test_alone(model, criterion, dataloader):
    running_loss = 0.0
    running_corrects = 0
            
    test_output_results = []
    test_output_expected = []
    
    model.eval()   # Set model to evaluate mode
    
    pbar = tqdm(enumerate(dataloader))
        # Iterate over data.
    for i, (inputs, labels) in pbar:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().view(dataloader.batch_size, 1))
        
        # statistics
        running_loss += loss.item()
            
        preds = torch.where(outputs > 0.5, torch.Tensor([1]).to(device), torch.Tensor([0]).to(device))
        running_corrects += torch.sum(preds == labels.float().view(dataloader.batch_size, 1)).item()
                
        test_output_results = np.concatenate([test_output_results, outputs.view(-1).cpu().numpy()])
        test_output_expected = np.concatenate([test_output_expected, labels.view(-1).cpu().numpy()])

        pbar.set_description('[{} {}/{}] Loss: {:.4f}, Acc: {:.4f}'.format('test', i, len(dataloader),
            running_loss / (i+1), running_corrects / ((i+1) * dataloader.batch_size)))
            
    test_output = {'expected': test_output_expected, 'results': test_output_results}

    return test_output

test_alone_needed = False;
if test_alone_needed == True:
    test_output = test_alone(model, criterion, dataloaders['test'])
    plot_roc_auc(test_output)

def predict(model, img_folder_path, transform=None):
    targets = []
    predictions = []
    
    model.eval()   # Set model to evaluate mode

    for filename in tqdm(os.listdir(img_folder_path)):
        image = Image.open(img_folder_path + filename)

        # Preprocess the image
        image_tensor = transform(image)

        # Add an extra batch dimension since pytorch treats all images as batches
        image_tensor = image_tensor.unsqueeze_(0)

        input = torch.autograd.Variable(image_tensor.to(device))

        # Predict the class of the image
        output = model(input)
        
        targets.append(filename.replace('.tif', ''))
        predictions.append(int(torch.where(output > 0.5, torch.Tensor([1]).to(device), torch.Tensor([0]).to(device)).item()))
        
    my_submission = pd.DataFrame({'id': targets, 'label': predictions})
    my_submission.to_csv('hplcd_submission.csv', index=False)

predict(model, test_dir, test_transformations)